# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:

print(y)

tensor([ 2,  4,  6,  8, 10])


In [2]:
import syft as sy

In [3]:
hook = sy.TorchHook(th)
print(hook)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [4]:
bob = sy.VirtualWorker(hook, id="bob")

In [5]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)
x

(Wrapper)>[PointerTensor | me:58504680661 -> bob:52351906319]

In [12]:
bob._objects

{52351906319: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #tensors:1>

In [14]:
x.id_at_location

52351906319

In [15]:
x.id

58504680661

In [16]:
x.owner

<VirtualWorker id:me #tensors:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:58504680661 -> bob:52351906319]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [59]:
# try this project here!
alice = sy.VirtualWorker(hook, id="Alice")

In [22]:
x = x.send(alice, bob)
x

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:19424764068 -> Alice:92466360908]
	-> (Wrapper)>[PointerTensor | me:12737529115 -> bob:79698311941]

In [23]:
bob._objects

{79698311941: tensor([1, 2, 3, 4, 5])}

In [24]:
alice._objects

{92466360908: tensor([1, 2, 3, 4, 5])}

In [26]:
x.id

72510706866

In [28]:
x.id_at_location

In [29]:
x.owner

<VirtualWorker id:me #tensors:0>

In [30]:
x = x.get()
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

# Lesson: Introducing Remote Arithmetic

In [31]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [32]:
x

(Wrapper)>[PointerTensor | me:41684303013 -> bob:40355439597]

In [33]:
y

(Wrapper)>[PointerTensor | me:88277618129 -> bob:35227002783]

In [34]:
z = x + y

In [35]:
z

(Wrapper)>[PointerTensor | me:3548825275 -> bob:86413464768]

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:45795014213 -> bob:308655355]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [40]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [41]:
z = (x + y).sum()

In [42]:
z.backward()

(Wrapper)>[PointerTensor | me:87395096889 -> bob:76261854094]

In [43]:
x = x.get()

In [44]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [45]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [6]:
# try this project here!
input = th.tensor([[0.,0],[0,1],[1,0],[1,1]], requires_grad = True)
labels = th.tensor([[0.],[0],[1],[1]], requires_grad = True)
print(input.shape, labels.shape)

torch.Size([4, 2]) torch.Size([4, 1])


In [7]:
input.send(bob)
labels.send(bob)

(Wrapper)>[PointerTensor | me:83700708127 -> bob:91622245623]

In [8]:
weights = th.tensor([[0.],[0]], requires_grad = True)
weights.send(bob)

(Wrapper)>[PointerTensor | me:8410791863 -> bob:87682709279]

In [11]:
for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - labels)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss)

tensor(0.3504, grad_fn=<SumBackward0>)
tensor(0.3361, grad_fn=<SumBackward0>)
tensor(0.3338, grad_fn=<SumBackward0>)
tensor(0.3334, grad_fn=<SumBackward0>)
tensor(0.3333, grad_fn=<SumBackward0>)
tensor(0.3333, grad_fn=<SumBackward0>)
tensor(0.3333, grad_fn=<SumBackward0>)
tensor(0.3333, grad_fn=<SumBackward0>)
tensor(0.3333, grad_fn=<SumBackward0>)
tensor(0.3333, grad_fn=<SumBackward0>)


In [14]:
weights

tensor([[0.3333],
        [0.3333]], requires_grad=True)

# Lesson: Garbage Collection and Common Errors


In [15]:
bob = bob.clear_objects()

In [16]:
bob._objects

{}

In [17]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [18]:
bob._objects

{1934792671: tensor([1, 2, 3, 4, 5])}

In [19]:
del x

In [20]:
bob._objects

{}

In [21]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [22]:
bob._objects

{52113535738: tensor([1, 2, 3, 4, 5])}

In [23]:
x = "asdf"

In [25]:
bob._objects

{}

In [26]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [27]:
x

(Wrapper)>[PointerTensor | me:63198741528 -> bob:77467025043]

In [28]:
bob._objects

{77467025043: tensor([1, 2, 3, 4, 5])}

In [29]:
x = "asdf"

In [30]:
bob._objects

{77467025043: tensor([1, 2, 3, 4, 5])}

In [31]:
del x

In [32]:
bob._objects

{77467025043: tensor([1, 2, 3, 4, 5])}

In [33]:
bob = bob.clear_objects()
bob._objects

{}

In [34]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [35]:
bob._objects

{6293395825: tensor([1, 2, 3, 4, 5])}

In [36]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [37]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:93457790121 -> bob:24934124765]
Tensor B: tensor([1, 1, 1, 1, 1])

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [47]:
z = x + y

In [48]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [50]:
from torch import nn, optim

In [52]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [53]:
# A Toy Model
model = nn.Linear(2,1)

In [54]:
optimizer = optim.SGD(params=model.parameters(), lr=0.1)

In [57]:
def train(iterations=20):
    for i in range(iterations):
        optimizer.zero_grad()

        pred = model(data)
        loss = ((pred - target)**2).sum()

        loss.backward()
        optimizer.step()

        print(loss)
        
train()

tensor(0.0006, grad_fn=<SumBackward0>)
tensor(0.0005, grad_fn=<SumBackward0>)
tensor(0.0003, grad_fn=<SumBackward0>)
tensor(0.0003, grad_fn=<SumBackward0>)
tensor(0.0002, grad_fn=<SumBackward0>)
tensor(0.0002, grad_fn=<SumBackward0>)
tensor(0.0001, grad_fn=<SumBackward0>)
tensor(8.7523e-05, grad_fn=<SumBackward0>)
tensor(6.6633e-05, grad_fn=<SumBackward0>)
tensor(5.0765e-05, grad_fn=<SumBackward0>)
tensor(3.8699e-05, grad_fn=<SumBackward0>)
tensor(2.9515e-05, grad_fn=<SumBackward0>)
tensor(2.2520e-05, grad_fn=<SumBackward0>)
tensor(1.7189e-05, grad_fn=<SumBackward0>)
tensor(1.3124e-05, grad_fn=<SumBackward0>)
tensor(1.0023e-05, grad_fn=<SumBackward0>)
tensor(7.6558e-06, grad_fn=<SumBackward0>)
tensor(5.8488e-06, grad_fn=<SumBackward0>)
tensor(4.4691e-06, grad_fn=<SumBackward0>)
tensor(3.4151e-06, grad_fn=<SumBackward0>)


In [61]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [62]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [63]:
dataset = [(data_bob, target_bob), (data_alice, target_alice)]

In [71]:
_data, _target = dataset[0]

In [72]:
_data.location

<VirtualWorker id:bob #tensors:4>

In [27]:
def train(iterations=20):
    
    model = nn.Linear(2,1)
    optimizer = optim.SGD(params=model.parameters(), lr=0.1)
    
    for _data, _target in dataset:
        #send model to local machines
        model = model.send(_data.location)

        #do normal training
        optimizer.zero_grad()
        pred = model(_data)
        loss = ((pred - _target)**2).sum()
        loss.backward()
        optimizer.step()

        #get your model back
        model = model.get()

        print(loss.get())

In [81]:
train()

tensor(0.1576, grad_fn=<SumBackward0>)
tensor(0.0849, grad_fn=<SumBackward0>)
tensor(0.0532, grad_fn=<SumBackward0>)
tensor(0.0340, grad_fn=<SumBackward0>)
tensor(0.0218, grad_fn=<SumBackward0>)
tensor(0.0139, grad_fn=<SumBackward0>)
tensor(0.0089, grad_fn=<SumBackward0>)
tensor(0.0057, grad_fn=<SumBackward0>)
tensor(0.0037, grad_fn=<SumBackward0>)
tensor(0.0024, grad_fn=<SumBackward0>)
tensor(0.0015, grad_fn=<SumBackward0>)
tensor(0.0010, grad_fn=<SumBackward0>)
tensor(0.0006, grad_fn=<SumBackward0>)
tensor(0.0004, grad_fn=<SumBackward0>)
tensor(0.0003, grad_fn=<SumBackward0>)
tensor(0.0002, grad_fn=<SumBackward0>)
tensor(0.0001, grad_fn=<SumBackward0>)
tensor(6.8911e-05, grad_fn=<SumBackward0>)
tensor(4.4540e-05, grad_fn=<SumBackward0>)
tensor(2.8840e-05, grad_fn=<SumBackward0>)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [86]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:Alice #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [87]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:53042493488 -> bob:20712794723]

In [88]:
x = x.send(alice)

In [89]:
x

(Wrapper)>[PointerTensor | me:22233720400 -> Alice:53042493488]

In [91]:
bob._objects

{20712794723: tensor([1, 2, 3, 4, 5])}

In [92]:
alice._objects

{53042493488: (Wrapper)>[PointerTensor | Alice:53042493488 -> bob:20712794723]}

In [93]:
y = x + x

In [94]:
y

(Wrapper)>[PointerTensor | me:26725694438 -> Alice:8355065877]

In [95]:
bob._objects

{20712794723: tensor([1, 2, 3, 4, 5]),
 4428130894: tensor([ 2,  4,  6,  8, 10])}

In [96]:
alice._objects

{53042493488: (Wrapper)>[PointerTensor | Alice:53042493488 -> bob:20712794723],
 8355065877: (Wrapper)>[PointerTensor | Alice:8355065877 -> bob:4428130894]}

In [97]:
jon = sy.VirtualWorker(hook, id="jon")

In [98]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [99]:
bob._objects

{84062988802: tensor([1, 2, 3, 4, 5])}

In [100]:
alice._objects

{92964887330: (Wrapper)>[PointerTensor | Alice:92964887330 -> bob:84062988802]}

In [101]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:92964887330 -> bob:84062988802]

In [102]:
bob._objects

{84062988802: tensor([1, 2, 3, 4, 5])}

In [103]:
alice._objects

{}

In [104]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [105]:
bob._objects

{}

In [106]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [107]:
bob._objects

{52053439181: tensor([1, 2, 3, 4, 5])}

In [108]:
alice._objects

{67741385188: (Wrapper)>[PointerTensor | Alice:67741385188 -> bob:52053439181]}

In [109]:
del x

In [110]:
bob._objects

{}

In [111]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [130]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:Alice #tensors:0>

In [113]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [114]:
bob._objects

{31780956895: tensor([1, 2, 3, 4, 5])}

In [115]:
alice._objects

{}

In [116]:
x.move(alice)

(Wrapper)>[PointerTensor | me:14703257209 -> Alice:14703257209]

In [123]:
bob._objects

{}

In [124]:
alice._objects

{}

In [131]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [132]:
bob._objects

{28176043966: tensor([1, 2, 3, 4, 5])}

In [133]:
alice._objects

{66911757626: (Wrapper)>[PointerTensor | Alice:66911757626 -> bob:28176043966]}

In [134]:
x.remote_get()

(Wrapper)>[PointerTensor | me:18093552849 -> Alice:66911757626]

In [135]:
bob._objects

{}

In [136]:
alice._objects

{66911757626: tensor([1, 2, 3, 4, 5])}

In [137]:
x.move(bob)

(Wrapper)>[PointerTensor | me:18093552849 -> bob:18093552849]

In [138]:
x

(Wrapper)>[PointerTensor | me:18093552849 -> bob:18093552849]

In [139]:
bob._objects

{18093552849: tensor([1, 2, 3, 4, 5])}

In [140]:
alice._objects

{}

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
#import syft as sy

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,))])

In [2]:
import syft as sy

hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
jon = sy.VirtualWorker(hook, id='jon')

In [43]:
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transform).federate((bob, alice)),# <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
                   batch_size=64, shuffle=True)

In [41]:
from torch import optim

model = nn.Sequential(nn.Linear(784, 128),
                     nn.ReLU(),
                     nn.Linear(128, 64),
                     nn.ReLU(),
                     nn.Linear(64, 10))

for param in model.parameters():
    param.requires_grad = True

optimizer = optim.SGD(params=model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [42]:
model

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
)

In [32]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [36]:
bob._objects

{941961408: tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]],
 
 
         [[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]],
 
 
         [[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]],
 
 
         ...,
 
 
         [[[-1., -1., -

In [40]:
images, labels = next(iter(federated_train_loader))
print(images[0].get(),labels[0].get())

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

In [46]:
epochs = 10

for epoch in range(epochs):
    
    running_loss = 0
    i = 0
    for data, target in federated_train_loader:
        
        #sending model    
        model = model.send(data.location)
        
        #normal training        
        data = data.view(data.shape[0], -1)
        
        optimizer.zero_grad()
        
        preds = model(data)
        print(preds.shape)
        loss = criterion(preds, target)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        
        #sending raw gradients to another worker
        model.move(bob)
        
        i += 1
    
    else:
        print(f"training loss for epoch #{epoch}: {running_loss/len(trainloader)}")
        
model = model.get()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn